In [3]:
import math

def haversine(coord1, coord2):
    # Haversine formula to calculate the distance between two points on the earth
    R = 6371  # Earth radius in km
    lat1, lon1 = coord1
    lat2, lon2 = coord2

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

In [4]:
import matplotlib.pyplot as plt

from random import randint 

from dadk.Optimizer import *

from tabulate import tabulate
import networkx as nx

class Optimizer_Demo_TSP_Model(OptimizerModel):
    ##################################################################################################################
    # constructor (defines GUI and test declaration)
    ##################################################################################################################
    def __init__(self, persistent_file="O_01_Optimizer_Demo_TSP.dao"):
        OptimizerModel.__init__(
            self,
            name = 'Demo_UIFunctions',
            load_parameter=[
                {'name':'N', 'type':'int_slider', 'value':12, 'min':2, 'max': 90, 
                 'label':'Size', 'description':'problem size', 'tags':['demo1', 'demo2']},
                {'name':'fixed_city', 'type':'int_slider', 'value':0, 'min':0, 'max':90, 
                 'label':'fixed city', 'description': '', 'tags':['demo1']},
                {'name':'fixed_time', 'type':'int_slider', 'value':0, 'min':0, 'max':90, 
                 'label':'fixed time', 'description':'', 'tags':['demo1']}
            ],            
            build_qubo_parameter=[
                {'name':'factor_rules', 'type':'float_slider', 'value':100.0, 'min':0.0, 'max':10**6,
                 'label':'factor rules'},
                {'name':'factor_distance', 'type':'float_slider', 'value':1.0, 'min':0.0, 'max':10**3,
                 'label':'factor distance'}
            ], 
            #
            # Define default values for the "Solve annealing" tab.
            
            #
            default_solver_parameter={
                'number_iterations':2000,
                'temperature_end':0.1,
                'offset_increase_rate':5.0},
            #
            # Define calculated values for the "Solve annealing" tab.
            
            #
            calculated_solver_parameter={'temperature_start': 50000},
            demo_solver_parameter=[{'name':'number_iterations', 'tags':['demo1', 'demo2']},
                                   {'name':'number_runs', 'tags':['demo1']},
                                   'number_replicas'
                                  ],
            
            
        )

    
    ##################################################################################################################
    # load method (defines content of tab Setup scenario)
    ##################################################################################################################
    def load(self, N, fixed_city=0, fixed_time=0, silent=False):
         # List of selected cities with their latitudes and longitudes
        city_coords = [
            (53.8486, -1.8383),  # shipley
            (53.8679, -1.9125),  # Keighley
            (53.8302, -1.9497),  # Haworth
            (53.8067, -1.8958),  # Denholme
            (53.7700, -1.8600),  # Queensbury
            (53.7220, -1.8632),  # Halifax
            (53.6458, -1.7850),  # Huddersfield
            (53.7220, -1.7125),  # Cleckheaton
            (53.7090, -1.7090),  # Liversedge
            (53.8508, -1.7594),  # Baildon
            (53.8758, -1.7125),  # Guiseley
            (53.9058, -1.6919),  # Otley
            (53.9240, -1.8225),  # Ilkley
            (53.9106, -1.7533),  # Burley
            (53.8917, -1.7308),  # Menston
            (53.8650, -1.6844),  # Yeadon
            (53.8536, -1.6844),  # Rawdon
            (53.7970, -1.8425),  # Thornton
            (53.7736, -1.6800),  # Tong Village 


           
        ]
        # store paramters in model attributes
        self.N = len(city_coords)
        self.fixed_city = min(fixed_city,self.N - 1)
        self.fixed_time = min(fixed_time,self.N - 1)
    # Store city coordinates
        self.cities = city_coords
        
        # Calculate distance table using Haversine formula
        self.distance_table = [[haversine(self.cities[c_1], self.cities[c_0]) for c_0 in range(self.N)] for c_1 in range(self.N)]
           
        
        
        # report actions to tab
        self.LOGGER.info(f'Model for  map with {self.N} cities created.')
        
        if not silent:
            print(f"""
Model for  map created:
    Number of cities: {self.N:3d}
    Fixed city:       {self.fixed_city:3d}
    Fixed time:       {self.fixed_time:3d}
""")
        
        with open("./O_01_distance_table.html", 'w') as detail:
            detail.write('<h1>Distances:</h1>' + tabulate(self.distance_table, range(N), showindex="always", tablefmt="html"))
        self.Optimizer.set_load_details_reference(url = "O_01_distance_table.html")
        
    ##################################################################################################################
    # build_qubo method (defines content of tab Build QUBO)
    ##################################################################################################################
    def build_qubo(self, factor_distance, factor_rules, silent=False):
        
        # variable bit as default are initialized with -1
        constant_bits = np.full((self.N, self.N), -1, np.int8)
        # all bits for fixed_time are set to constant 1
        constant_bits[self.fixed_time, :] = 0
        # all bits for fixed city are set to constant 0
        constant_bits[:, self.fixed_city] = 0
        # bit for fixed_time in fixed_city is set to constant 1
        constant_bits[self.fixed_time, self.fixed_city] = +1                
        # square array x with constant bits is created as VarShapeSet            
        self.qubo_vs = VarShapeSet(BitArrayShape('x', (self.N, self.N), constant_bits=constant_bits, axis_names=['Time', 'City']))        
                        
        #-P_time-------------------------------------------------------------------------------
        
        self.P_time = BinPol(self.qubo_vs)   
        for t in range(self.N):
            self.P_time += (1 - BinPol.sum(Term(1, (('x', t, c),), self.qubo_vs) for c in range(self.N))) ** 2
        self.P_time *= factor_rules
        
        #-P_city-------------------------------------------------------------------------------

        self.P_city = factor_rules *  BinPol.sum(
            (1 - BinPol.sum(Term(1, (('x', t, c),), self.qubo_vs) for t in range(self.N))) ** 2 
            for  c in range(self.N))

        #-P_distance---------------------------------------------------------------------------
                        
        self.P_distance = factor_distance * BinPol.sum(Term(self.distance_table[c_0][c_1], (('x', t, c_0), ('x', (t + 1) % self.N, c_1)), self.qubo_vs)
                                                       for c_0 in range(self.N) for c_1 in range(self.N) if c_0 != c_1 for t in range(self.N))

        #-HQ-----------------------------------------------------------------------------------

        self.HQ = self.P_time + self.P_city + self.P_distance
        
        # calculate and set start temperature as standard deviation of QUBO energy
        recommended_temperature_start = round(0.1 * np.std([self.HQ.compute([randint(0, 1) for _ in range(self.N**2)]) for case in range(20)]), 3)
        self.Optimizer.set_calculated_solver_parameter(temperature_start=recommended_temperature_start)
        
        if not silent:
            print(f"""
    Polynomial created 
    Number of bits:               {self.HQ.N: 5d}
    Calculated start_temperature: {recommended_temperature_start: 9.3f}""")
    
    ##################################################################################################################
    # prep_result method (evaluates result of annealing defines content of tab Solve annealing)
    ##################################################################################################################
    def prep_result(self, solution_list:SolutionList, silent=False):
    
        configuration = solution_list.min_solution.configuration
        
        self.configuration = configuration # <-- best solution
        self.start_table = None
        self.route = None
              
        if not silent:
            e_HQ = self.HQ.compute(self.configuration)
            e_time = self.P_time.compute(self.configuration)
            e_city = self.P_city.compute(self.configuration)
            e_distance = self.P_distance.compute(self.configuration)
            
            print('Energy of QUBO / parts in best solution (P_time and P_city should be zero)')
            print("HQ         = %10.2F" % e_HQ)
            print("P_time     = %10.2F" % e_time)
            print("P_city     = %10.2F" % e_city)
            print("P_distance = %10.2F" % e_distance)
            print("")
            if e_time == 0 and e_city == 0:

                self.start_table = solution_list.min_solution.extract_bit_array('x')
                self.route = [np.argmax(self.start_table.data[t,:]) for t in range(self.N)]

                distances = [self.distance_table[self.route[t]][self.route[(t + 1) % self.N]] for t in range(self.N)]
                self.output_table = tabulate(zip(range(self.N),
                                                 self.route,
                                                 self.route[1:] + self.route[:1],
                                                 distances,
                                                 np.cumsum(distances)
                                                ),
                                             ['t', 'from', 'to', 'distance', 'total'], tablefmt="html")
                display(HTML(self.output_table))
            else:
                print("invalid solution, try again!")

        samples = []
        for solution in solution_list.get_solution_list():
            configuration = solution.configuration

            start_table = solution.extract_bit_array('x')
            route = [min(c if start_table.data[t][c] else self.N for c in range(self.N)) 
                     for t in range(self.N)]
            distance = sum(self.distance_table[route[t]][route[(t + 1) % self.N]] 
                           for t in range(self.N))

            samples.append({"solution":solution, "route":route, "distance":distance})

        for sample in samples:
            print("frequency=%d  route=%s  distance=%.2F" % 
                  (sample["solution"].frequency, str(sample["route"]), sample["distance"]))
           
    ##################################################################################################################
    # report method (defines content of Report tab)
    ##################################################################################################################
    def report(self, silent=False):
        
        if self.start_table is not None:
            self.start_table.draw(figsize=(6.0, 6.0))
            
    def report_title(self):
        return "Bit map report"
    
    ##################################################################################################################
    # report1 method (defines content of additional Report tab)
    ##################################################################################################################
    def report1(self, silent=False):
        print('This is the best found roundtrip:')
        if self.start_table is not None:
            display(HTML(self.output_table))
            
    def report1_title(self):
        return "Route report"
    
    ##################################################################################################################
    # draw method (defines content of visualization tab)
    ##################################################################################################################
    def draw(self):
        if self.start_table is not None and self.route is not None:
            self.graph = nx.Graph()

            nodes={}
            for i in range(self.N):
                nodes[i] = str(i);
                self.graph.add_node(nodes[i], pos=(self.cities[i][0], self.cities[i][1]))

            path = [[self.route[t], self.route[(t + 1) % self.N]] for t in range(self.N)]

            for i in range(self.N):
                for j in range(i+1, self.N):
                    if [i, j] in path or [j, i] in path: 
                        self.graph.add_edge(nodes[min(i,j)], nodes[max(i,j)], weight=10)
                    else:
                        self.graph.add_edge(nodes[min(i,j)], nodes[max(i,j)], weight=1)

            fig = plt.figure(num='Graph (created at %s)' % datetime.now(), figsize=(8.0, 8.0))

            pos=nx.get_node_attributes(self.graph,'pos')

            # nodes
            nx.draw_networkx_nodes(self.graph, pos, nodelist=self.graph.nodes, node_size=100, node_color="red")

            # edges        
            edges_bold = [(u, v) for (u, v, d) in self.graph.edges(data=True) if d['weight'] == 10]
            nx.draw_networkx_edges(self.graph, pos, edgelist=edges_bold, width=3, edge_color="black")

            edges_thin = [(u, v) for (u, v, d) in self.graph.edges(data=True) if d['weight'] == 1]
            nx.draw_networkx_edges(self.graph, pos, edgelist=edges_thin, width=1, alpha=0.1, edge_color="black")

            # labels
            nx.draw_networkx_labels(self.graph, pos, font_size=6, font_family='sans-serif')

            plt.axis('off')
            plt.show()
        
    
        
        
optimizer = Optimizer(Optimizer_Demo_TSP_Model())


Output()

Optimizer(children=(OptimizerTab(children=(VBox(children=(HBox(layout=Layout(width='100%')), WidgetGui(childre…